# How to Create Tools

Tools is something that agent can use. For example function to browser, database query, or time getter. 

References: [How to Custom Tools](https://python.langchain.com/v0.2/docs/how_to/custom_tools/)

This is common way to build tools:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [15]:
from langchain_core.tools import StructuredTool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Create a function
def add(a: int, b: int):
    return a + b

# Create a tool
add_tool = StructuredTool.from_function(
    name="Add", 
    func=add,  
    description="Useful for adding two numbers",
)

tools = [add_tool]

# Describe model
model = ChatOpenAI(model="gpt-3.5-turbo")

# Create agent from tools and model
agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [('human', 'Hi, what is 2 + 2?')]})

response["messages"]

[HumanMessage(content='Hi, what is 2 + 2?', id='e3a55843-eaa2-4e50-b3e3-390786c7b64d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yjebeuTYiZAmwDFUZWzqXc7j', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'Add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 57, 'total_tokens': 74}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ebf39cbc-dc29-451d-b44f-259129a0d96a-0', tool_calls=[{'name': 'Add', 'args': {'a': 2, 'b': 2}, 'id': 'call_yjebeuTYiZAmwDFUZWzqXc7j', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 17, 'total_tokens': 74}),
 ToolMessage(content='4', name='Add', id='3a06a03a-3ad6-4853-bb04-a518b7b0734d', tool_call_id='call_yjebeuTYiZAmwDFUZWzqXc7j'),
 AIMessage(content='2 + 2 is equal to 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_t

To build tools, we need specify:

| **Name**        | **Type**              | **Description**                                                                                                                                           |
|-----------------|-----------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| `name`          | `str`                 | Must be unique within a set of tools provided to an LLM or agent.                                                                                         |
| `description`   | `str`                 | Describes what the tool does. Used as context by the LLM or agent.                                                                                         |
| `args_schema`   | `Pydantic BaseModel`  | Optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.                         |

Langchain has three ways to create a tool:

[1. Tool from functions](#1)
[2. Tool from LangChain Runnables](#2)
[3. Tool By sub-classing from BaseTool](#3)

## 1 - Create Tool from Functions

### a. Using `StructuredTool.from_function` Class

- This is most strightforward and flexible way to create a tool.
- Here, we can specify the sync and async function and args scheme

Reference: [`StructuredTool`](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.StructuredTool.html#langchain_core.tools.StructuredTool)

In [42]:
from langchain_core.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field

# Specify arg scheme
class CalculatorArgsScheme(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

# Define sync and async functions
def multiply(a: int, b: int) -> int:
    """Multiply two numbers sync."""
    return a * b

async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers async."""
    return a * b

# We can define syn and async functions using StructuredTool
calculator = StructuredTool.from_function(func=multiply, 
                                          coroutine=amultiply, 
                                          args_schema=CalculatorArgsScheme,
                                          description="Multiply two numbers")

In [43]:
print(calculator.description)
print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 5}))
calculator.args_schema.schema()

Multiply two numbers
6
10


{'title': 'CalculatorArgsScheme',
 'type': 'object',
 'properties': {'a': {'title': 'A',
   'description': 'first number',
   'type': 'integer'},
  'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}},
 'required': ['a', 'b']}

### b. Using `@tool` decorator

- This is more simple way, yet more limited

References: [@tool](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.tool.html)

In [44]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

Let's inspect some of the attributes associated with the tool.

In [47]:
# @tool decorator will produce StructuredTool object
print(type(multiply))

print(multiply.name)
print(multiply.description)
multiply.args_schema.schema()

<class 'langchain_core.tools.StructuredTool'>
multiply
Multiply two numbers.


{'title': 'multiplySchema',
 'description': 'Multiply two numbers.',
 'type': 'object',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b']}

By default, langchain will create async function from the function. But we cant modify it.

In [46]:
await multiply.ainvoke({"a": 22, "b": 2})

44

To specify args scheme, there is two ways: First, using annotated class

In [50]:
from typing import Annotated, List

@tool
def multiply_by_max(
    a: Annotated[str, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)

multiply_by_max.args_schema.schema()

{'title': 'multiply_by_maxSchema',
 'description': 'Multiply a by the maximum of b.',
 'type': 'object',
 'properties': {'a': {'title': 'A',
   'description': 'scale factor',
   'type': 'string'},
  'b': {'title': 'B',
   'description': 'list of ints over which to take maximum',
   'type': 'array',
   'items': {'type': 'integer'}}},
 'required': ['a', 'b']}

Second, using `parse_docstring=True`

In [51]:
@tool(parse_docstring=True)
def foo(bar: str, baz: int) -> str:
    """The foo.

    Args:
        bar: The bar.
        baz: The baz.
    """
    return bar

foo.args_schema.schema()

{'title': 'fooSchema',
 'description': 'The foo.',
 'type': 'object',
 'properties': {'bar': {'title': 'Bar',
   'description': 'The bar.',
   'type': 'string'},
  'baz': {'title': 'Baz', 'description': 'The baz.', 'type': 'integer'}},
 'required': ['bar', 'baz']}

## Create Tools From Runnable <div id="2"></div>

In [52]:
from langchain_core.language_models import GenericFakeChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("human", "Hello. Please respond in the style of {answer_style}.")]
)

# Placeholder LLM
llm = GenericFakeChatModel(messages=iter(["hello matey"]))

chain = prompt | llm | StrOutputParser()

as_tool = chain.as_tool(
    name="Style responder", description="Description of when to use tool."
)
as_tool.args

d:\Document\Machine_Learning_Journey\Projects\Langchain\learn-langchain-from-scratch\.venv\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


{'answer_style': {'title': 'Answer Style', 'type': 'string'}}

## Sub-Class BaseModel <div id="2"></div>

- This is the most flexible method, it provides the largest degree of control, at the expense of more effort and code.
- Basically, all tools must be implemented `BaseModel` class

In [53]:
from typing import Optional, Type

from langchain.pydantic_v1 import BaseModel
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If the sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())